# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

[Write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) for this project

[Rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

- - -
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle


# Paths to files with dataset
training_file   = 'traffic-signs-data/train.p'
validation_file = 'traffic-signs-data/valid.p'
testing_file    = 'traffic-signs-data/test.p'

with open(training_file, mode = 'rb') as f:
    train = pickle.load(f)
with open(validation_file, mode = 'rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode = 'rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Data summary is represented below.
For this task used python, numpy and pandas methods to calculate the data summary.<br>
The [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) is useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
import pandas as pd


# Number of training examples
# shape = (34799, 32, 32, 3)
n_train = X_train.shape[0]

# Number of testing examples
# shape = (12630, 32, 32, 3)
n_test = X_test.shape[0]

# Shape of an traffic sign image
# Taking first image from testing set and getting it's shape
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
classes_file = 'signnames.csv'
with open(classes_file, mode = 'rb') as f:
    classes = pd.read_csv(f)
n_classes = classes.shape[0]

print("Number of training examples = {}".format(len(X_train)))
print("Number of testing examples = {}".format(len(X_test)))

print("Image data shape = {}".format(image_shape))
print("Number of classes = {}".format(n_classes))

### Include an exploratory visualization of the dataset

Visualization of the German Traffic Signs Dataset using the pickled files: plotting traffic sign images, plotting the count of each sign in different datasets.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

In [ ]:
# Data exploration visualization code goes here.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline


# Get sign description by it's index (label)
# 'index' - sign index
def get_sign_label(index):
    # Looking for sign with proper index
    for sign_class in sign_dict:
        if sign_class[0] == index:
            return sign_class[1]
    return('Index not found')

# Count how many times class represented inside dataset
# 'index' - sign index
# 'dataset' - dictionary with sign labels
def count_class(index, dataset):
    count = 0
    for c in dataset:
        if c == index:
            count += 1
    return count

# Find picture index in validation set 
# Validation set is small and search operation will be faster
def find_pic_id(index):
    for i in range(len(y_valid)):
        if y_valid[i] == index:
            return i
    # If result not found
    return None


# Convert Pandas data frame to Numpy array
sign_dict = classes.as_matrix()

# Figure size
plt.figure(figsize=(20, 25))

# Pass through the all classes
for num_model in range(n_classes):
    # Get sign class Id
    sign_class_id = sign_dict[num_model, 0]
    
    # Count number of classes in training set
    class_count_train = count_class(sign_class_id, y_train)
    # Count number of classes in validation set
    class_count_valid = count_class(sign_class_id, y_valid)
    # Count number of classes in test set
    class_count_test = count_class(sign_class_id, y_test)

    # Find picture id in validation set
    pic_id = find_pic_id(sign_class_id)
   
    # Plot subplots
    # Each subplot - one sign class with title
    plt.subplot(9, 5, num_model + 1)
    title = "{0} \nTraining set: {1}\n Validation set: {2}\n Test set: {3}".format(
        get_sign_label(y_valid[pic_id]),
        class_count_train,
        class_count_valid,
        class_count_test)
    plt.title(title, wrap=True)
    plt.imshow(X_valid[pic_id])
    
# Auto layout
plt.tight_layout()
#plt.savefig('01-dataset-summary.png')

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs.<br>
Training and testint the model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point.

With the LeNet-5 solution, I've got a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission.

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Apply preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set
1. Shuffle the training data.<br>
2. AUGMENTATION: Increase 2 times the training set. Second part of training set will be a rotated (-10 degrees counterclock-wise) and zoomed (ratio = 1.25) images from first part.<br>
3. Apply grayscale filter for all datasets.<br>
4. Apply normalization filter (cv.NORM_MINMAX) for all datasets.

In [ ]:
from sklearn.utils import shuffle
import numpy as np
import cv2


X_train, y_train = shuffle(X_train, y_train)

# Generate more training examples - AUGMENTATION
def right_rotate(X_data, y_data):
    bat_x = []
    bat_y = []
    angle = 10
    for i in range(0, len(X_data)):
        image = X_data[i]
        num_rows, num_cols = image.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), 10, 1.25)
        img_rotation = cv2.warpAffine(image, rotation_matrix, (num_cols, num_rows))
        bat_x.append(img_rotation)
        bat_y.append(y_data[i])
    return bat_x, bat_y


# Convert RGB to Grayscale:
def convert_to_grayscale(X_data):
    bat = []
    for i in range(0, len(X_data)):
        image = cv2.cvtColor(X_data[i], cv2.COLOR_RGB2GRAY)
        bat.append(image)
    X_data = np.reshape(bat, (-1, 32, 32, 1))
    return X_data

# Normalization:
def normalize(X_data):
    bat = []
    for i in range(0, len(X_data)):
        image = X_data[i]
        cv2.normalize(image, image, 0, 10, norm_type=cv2.NORM_MINMAX)
        bat.append(image)
    X_data = bat
    return X_data


# Control #1 (before filtering)
# Plot result
plt.figure(figsize=(10, 10))
plt.subplot(1, 5, 1)
plt.title("Original image", wrap=True)
plt.imshow(X_train[0])

# Rotate and scale
X_train_r, y_train_r = right_rotate(X_train, y_train)
plt.subplot(1, 5, 2)
plt.title("Rotation:\n -10 deg.", wrap=True)
plt.imshow(X_train_r[0])
m_X = []
m_y = []
m_X.extend(X_train)
m_X.extend(X_train_r)
m_y.extend(y_train)
m_y.extend(y_train_r)
X_train = m_X
y_train = m_y
print("Number of augmented training examples = {}".format(len(X_train_r)))

# Filter #1 - convert RGB to Grayscale
X_train = convert_to_grayscale(X_train)
X_valid = convert_to_grayscale(X_valid)
X_test = convert_to_grayscale(X_test)
# Control #2 (after filter #1)
plt.subplot(1, 5, 3)
plt.title("Filter 1:\n RGB-to-Grayscale", wrap=True)
plt.imshow(np.reshape(X_train[0], (32, 32)), cmap='gray')

# Filter #2 - normalize
X_train = normalize(X_train)
X_valid = normalize(X_valid)
X_test = normalize(X_test)
# Control #3
plt.subplot(1, 5, 4)
plt.title("Filter 2:\n Normalize", wrap=True)
plt.imshow(np.reshape(X_train[0], (32, 32)), cmap='gray')

# Control #4
plt.subplot(1, 5, 5)
plt.title("Example from\n augmented training set", wrap=True)
plt.imshow(np.reshape(X_train[34799], (32, 32)), cmap='gray')

plt.tight_layout()
print("Number of training examples = {}".format(len(X_train)))
#plt.savefig('02-dataset-preprocessing.png')

### Model Architecture
The EPOCHS and BATCH_SIZE values affect the training speed and model accuracy.

In [ ]:
import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 128

This solution based on LeNet-5 architecture (modification).

### Input
The LeNet architecture accepts a 32x32x<b>C</b> image as input, where <b>C</b> is the number of color channels.<br>
By default dataset images are <b>RGB</b>, <b>C = 3</b> in this case.<br>
After an <b>RGB to grayscale</b> transform, <b>C = 1</b>.

### Architecture
<table>
<tr><td>**Layer**</td>
<td>**Description**</td></tr>
<tr><td>**Layer 1: Convolutional**</td>
<td>Output shape is 28x28x6</td></tr>
<tr><td>**Activation: ReLU**</td>
<td>ReLU - a rectified linear unit</td></tr>
<tr><td>**Pooling**</td>
<td>Output shape is 14x14x6</td></tr>
<tr><td>**Layer 2: Convolutional**</td>
<td>Output shape is 10x10x16</td></tr>
<tr><td>**Activation: ReLU**</td>
<td>ReLU - a rectified linear unit</td></tr>
<tr><td>**Pooling**</td>
<td>Output shape is 5x5x16</td></tr>
<tr><td>**Flatten**</td>
<td>Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do it is using<br> <i>tf.contrib.layers.flatten</i>. Transform to vector with 400 output elements.</td></tr>
<tr><td>**Layer 3: Fully connected**</td>
<td> 120 outputs</td></tr>
<tr><td>**Activation: ReLU**</td>
<td>ReLU - a rectified linear unit</td></tr>
<tr><td>**Dropout operation**</td>
<td></td></tr>
<tr><td>**Layer 4: Fully connected**</td>
<td>84 outputs</td></tr>
<tr><td>**Activation: ReLU**</td>
<td>ReLU - a rectified linear unit</td></tr>
<tr><td>**Dropout operation**</td>
<td></td></tr>
<tr><td>**Layer 5: Fully Connected (Logits)**</td>
<td>43 outputs</td></tr>
</table>


### Output
Return the result of the 2-nd fully conected layer.

In [ ]:
from tensorflow.contrib.layers import flatten

def LeNet(x):
    # Hyperparameters.Used for tf.truncated_normal: randomly define variables 
    # for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Layer 1. Convolutional
    # Input = 32x32x3
    # output_height = (input_height - filter_height + Padding)/Stride = (32 - 5 + 1) / 1 = 28
    # output_width = (input_width - filter_width + Padding)/Stride = (32 - 5 + 1) / 1 = 28
    # Output = 28x28x6
    # filter 'shape' = (height, width, input_depth, output_depth)
    conv1_w = tf.Variable(tf.truncated_normal(shape = (5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    # strides = [batch, height, width, depth]
    conv1 = tf.nn.conv2d(x, conv1_w, strides = [1, 1, 1, 1], padding = 'VALID', name='conv1') + conv1_b
    
    # Activation
    conv1 = tf.nn.relu(conv1, name='conv1_relu')
    
    # Pooling
    # Input = 28x28x6
    # output_height = ((input_height - filter_height) / Stride) + 1 = (28 - 2 / 2) + 1 = 14
    # output_width = ((input_height - filter_height) / Stride) + 1 = (28 - 2 / 2) + 1 = 14
    # Output = 14x14x6
    # Kernel size - 'ksize' = [batch, height, width, depth]
    conv1 = tf.nn.max_pool(conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID', name='conv1_pool')
    
    # Layer 2. Convolutional
    # Input = 14x14x6 -> Output = 10x10x16
    conv2_w = tf.Variable(tf.truncated_normal(shape = (5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(conv1, conv2_w, strides = [1, 1, 1, 1], padding = 'VALID', name='conv2') + conv2_b
    
    # Activation
    conv2 = tf.nn.relu(conv2, name='conv2_relu')
    
    # Pooling
    # Input = 10x10x16 -> Output = 5x5x16
    conv2 = tf.nn.max_pool(conv2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID', name='conv2_pool')
    
    # Flatten
    # Input = 5x5x16 -> Output = 400
    fc0 = flatten(conv2)
    
    # Layer 3. Fully connected
    # Input = 400 -> Output = 120
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc0, fc1_w, name='fc1') + fc1_b
    
    # Activation
    fc1 = tf.nn.relu(fc1, name='fc1_relu')
    # Dropout
    fc1 = tf.nn.dropout(fc1, keep_prob, name='fc1_dropout')
    
    # Layer 4. Fully connected
    # Input = 120 -> Output = 84
    fc2_w = tf.Variable(tf.truncated_normal(shape = (120, 84), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1, fc2_w, name='fc2') + fc2_b
    
    # Activation
    fc2 = tf.nn.relu(fc2, name='fc2_relu')
    # Dropout
    fc2 = tf.nn.dropout(fc2, keep_prob, name='fc2_dropout')
    
    # Layer 5. Fully connected
    # Input = 84 -> Output = 43 (Number of classes)
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84, 43), mean = mu, stddev = sigma))
    fc3_b = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_w, name='logits') + fc3_b
    
    return logits

### Train, Validate and Test the Model
A validation set can be used to assess how well the model is performing. A <b>low accuracy</b> on the training and validation
sets imply <b>underfitting</b>. A <b>high accuracy</b> on the training set but low accuracy on the validation set implies <b>overfitting</b>.

### Features and Labels

<b>x</b> is a placeholder for a batch of <b>input images</b><br>
<b>y</b> is a placeholder for a batch of <b>output labels</b><br>
<b>keep_prob</b> is a probability to keep units during dropout operation

In [ ]:
# Placeholder to store input batches
# (BATCH_SIZE, height, width, depth)
x = tf.placeholder(tf.float32, (None, 32, 32, 1))

# Placeholder for labels
y = tf.placeholder(tf.int32, (None))

# One-hot encoding of labels
one_hot_y = tf.one_hot(y, 43)

# Probability to keep units
keep_prob = tf.placeholder(tf.float32)

### Training Pipeline

In [ ]:
# Learning rate
rate = 0.001

# Pass input data to the LeNet function
logits = LeNet(x)

# Compare logits to the ground-truth labels and calculate the cross entropy
# Cross entopy is a measure how different the logits
# are from the ground-truth training labels
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = one_hot_y)
# Average the cross entropy from all the training images
loss_operation = tf.reduce_mean(cross_entropy)
# Use Adam algorithm (alternative of stochastic gradient descent)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
# Optimizer uses backpropagation to update the network and minimize training loss
training_operation = optimizer.minimize(loss_operation)

### Model Evaluation
Evaluate how well the <b>loss</b> and <b>accuracy</b> of the model for a given dataset.

In [ ]:
# Compare the logit prediction to the one hot encoded ground-truth label
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
# Calculate the model's overall accuracy by averaging the individual prediction accuracies
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict = {x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Train the Model
Calculate and report the accuracy on the training and validation set.<br>
1. Run the training data through the training pipeline to train the model.<br>
2. <b>Before</b> each epoch, shuffle the training set.<br>
3. <b>After</b> each epoch, measure the loss and accuracy of the validation set.<br>
4. <b>Save</b> the model after training.

In [ ]:
# Create the TensorFlow session and Initialize the variables
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training in progress...")
    print()
    for i in range(EPOCHS):
        # Shuffle the training data to ensure that trainint isn't biased
        # by the order of the images
        X_train, y_train = shuffle(X_train, y_train)
        # Break training data into batches and train the model on the each batch
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict = {x: batch_x, y: batch_y, keep_prob: 0.75})
        
        # In the end of each EPOCH evaluate the model on validation data
        training_accuracy = evaluate(X_train, y_train)
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {0} ...".format(i + 1))
        print("Training Accuracy = {:.5f}".format(training_accuracy))
        print("Validation Accuracy = {:.5f}".format(validation_accuracy))
        print()
    
    # Save the model
    try:
        saver
    except NameError:
        saver = tf.train.Saver()
    saver.save(sess, 'lenet_sign_classifier')
    print("Model saved")

## Evaluate the Model
Evaluate the performance of the model on the test set.

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.5f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how model is working, download at least five pictures of German traffic signs from the web (8 signs downloaded) and use the model to predict the traffic sign type.

The file `signnames.csv` is useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2

X_web = []
path = "web-sign-examples/"
web_img_list = os.listdir(path)
# Read all images from web-sign-examples folder
for img_name in web_img_list:
    if (img_name[-3:] == 'jpg'): 
        image_BGR = cv2.imread(path + img_name)
        image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
        # Resize all images to 32x32x3
        resize_image = cv2.resize(image_RGB, (32, 32)) 
        X_web.append(resize_image)

# Add class labels for images
y_web = [18, 9, 0, 26, 14, 12, 13, 33]

# Plot images with class names
# Each subplot - one sign class with title
plt.figure(figsize=(8, 8))
for i in range(len(X_web)):
    plt.subplot(4, 4, i + 1)
    title = get_sign_label(y_web[i])
    plt.title(title, wrap=True)
    plt.imshow(X_web[i])
plt.tight_layout()
#plt.savefig('03-web-dataset.png')

### Predict the Sign Type for Each Image
#### Preprocessing

In [ ]:
plt.figure(figsize=(6, 6))
# Original image from web set
plt.subplot(1, 3, 1)
plt.title("Original image", wrap=True)
plt.imshow(X_web[0])

# Filter #1 - convert RGB to Grayscale
X_web = convert_to_grayscale(X_web)
# Plot image after filter #1
plt.subplot(1, 3, 2)
plt.title("Filter 1:\n RGB-to-Grayscale", wrap=True)
plt.imshow(np.reshape(X_web[0], (32, 32)), cmap='gray')

# Filter #2 - normalize
X_web = normalize(X_web)
# Plot image after filter #2
plt.subplot(1, 3, 3)
plt.title("Filter 2:\n Normalize", wrap=True)
plt.imshow(np.reshape(X_web[0], (32, 32)), cmap='gray')
plt.tight_layout()
#plt.savefig('04-web-dataset-preprocessing.png')

### Run the predictions and use the model to output the prediction for each image.
<i>Images were pre-processed with the same pre-processing pipeline used earlier.</i>

In [ ]:
from IPython.core.display import display, HTML


softmax = tf.nn.softmax(logits)
pred = tf.nn.top_k(softmax, 1)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    logits_web = sess.run(logits, feed_dict={x: X_web, keep_prob:1})
    
    # Print Real Labels and Predictions
    table_body = ""
    for i in range(len(logits_web)):
        real_title = get_sign_label(y_web[i])
        pred_title = get_sign_label(np.argmax(logits_web[i]))
        is_correct = (y_web[i] == np.argmax(logits_web[i]))
        table_header = "<table><tr><td><b>Original title</b></td><td><b>Predicted title</b> \
                        </td><td><b>Prediction result</b></td></tr>"
        table_body += "<tr><td>{0}</td><td>{1}</td><td>{2}</td></tr>".format(real_title, pred_title, is_correct)
        table_footer = "</table>"
    display(HTML(
        table_header + table_body + table_footer
    ))

### Analyze Performance
Calculate the accuracy for new images.<br>
For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    web_accuracy = evaluate(X_web, y_web)
    print("Web Accuracy = {:.3f}".format(web_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions
### on the German traffic sign images found on the web. 
softmax = tf.nn.softmax(logits)
pred = tf.nn.top_k(softmax, 5)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    result = sess.run(pred, feed_dict={x: X_web, keep_prob: 1.})
    predictions  = result.values
    predictionIndicies  = result.indices
    np.set_printoptions(precision=5, suppress=True)
    
    print("Predictions:\n {}".format(predictions))
    print("Labels:\n {}".format(predictionIndicies))

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

In [ ]:
# Visualize the network's feature maps.

# 'image_input': the test image being fed into the network to produce the feature maps

# 'tf_activation': should be a tf variable name used during your training procedure that 
#  represents the calculated state of a specific weight layer

# 'activation_min/max': can be used to view the activation contrast in more detail, 
#  by default matplot sets min and max to the actual min and max values of the output

# 'plt_num': used to plot out multiple different weight feature map sets on the same block, 
#  just extend the plt number for each new feature map entry


def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1, plt_num=2):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess, feed_dict={x: image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15, 15))
    for featuremap in range(featuremaps):
        plt.subplot(6, 8, featuremap + 1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")
        plt.tight_layout()

In [ ]:
with tf.Session() as sess:
    # Convolution (layer 1 after 'tf.nn.conv2d' operation)
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    conv1 = sess.graph.get_tensor_by_name('conv1:0')
    outputFeatureMap(X_web, conv1)

In [ ]:
with tf.Session() as sess:
    # ReLU-activation (layer 1 after 'tf.nn.relu' operation)
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    conv1_relu = sess.graph.get_tensor_by_name('conv1_relu:0')
    outputFeatureMap(X_web, conv1_relu)

In [ ]:
with tf.Session() as sess:
    # Max-pooling (layer 1 after 'tf.nn.max_pool' operation)
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    conv1_pool = sess.graph.get_tensor_by_name('conv1_pool:0')
    outputFeatureMap(X_web, conv1_pool)

### Question 9
How you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images?

### Answer:
Let's have a look for 3 examples:

<b>1. Convolution</b> (layer 1 after 'tf.nn.conv2d' operation)<br>
It's possible to see shape of sign, but there are many background noizes.

<b>2. ReLU-activation</b> (layer 1 after 'tf.nn.relu' operation)<br>
Noize level (from backgroung objects) significantly reduced. This demonstrates that the CNN learned to detect useful traffic sign features on its own.

<b>3. Max-pooling</b> (layer 1 after 'tf.nn.max_pool' operation)<br>
Image size reduced, but important features are available